In [1]:
import os
import pandas as pd

# Plotly and cufflinks for data analysis
from plotly.offline import init_notebook_mode, iplot
import cufflinks as cf
init_notebook_mode(connected=True) 

cf.set_config_file(theme='ggplot')
cf.go_offline()

# Matplotlib for drawing the court
from matplotlib.patches import Ellipse
import plotly.express as px
import numpy as np
from matplotlib import pyplot as plt
from math import pi
import plotly.graph_objects as go
%matplotlib inline

In [2]:
INPUT_DIR = './Tennis_data'
def load_data():
    return [pd.read_csv(os.path.join(INPUT_DIR, file_), index_col=0) for file_ in os.listdir(INPUT_DIR)]
        
rallies, points, events, serves = load_data()

In [3]:
# events
events.head()

,rallyid,frameid,strokeid,hitter,receiver,isserve,serve,type,stroke,hitter_x,hitter_y,receiver_x,receiver_y,time
0,1,70877,1,Djokovic,Nadal,True,first,serve,forehand,6.50,-0.24,1.03,27.44,0.00
1,1,70900,2,Nadal,Djokovic,False,first,slice,backhand,0.05,25.59,6.17,1.11,0.92
2,1,70950,3,Djokovic,Nadal,False,first,topspin,forehand,1.42,2.33,4.75,26.45,2.92
3,2,71488,1,Djokovic,Nadal,True,first,serve,forehand,4.48,-0.01,7.90,26.83,24.44
4,3,71796,1,Djokovic,Nadal,True,second,serve,forehand,4.48,-0.37,7.88,26.47,36.76


In [4]:
points.head()

,rallyid,server,returner,winner,reason,serve,strokes,totaltime,x,y,score
0,1,Djokovic,Nadal,Djokovic,winner,first,3,0.92,1.92,21.96,"0:0, 15:0"
2,3,Djokovic,Nadal,Djokovic,out,second,4,4.16,3.33,-0.39,"0:0, 30:0"
4,5,Djokovic,Nadal,Djokovic,ace,second,2,0.40,1.62,17.18,"0:0, 40:0"
5,6,Djokovic,Nadal,Djokovic,net,first,10,10.40,6.41,12.78,"1:0, 0:0"
7,8,Nadal,Djokovic,Djokovic,out,second,7,7.20,0.50,18.67,"1:0, 15:0"


In [5]:
temp = events.merge(right=points, how='left', left_on='rallyid', right_on='rallyid')

In [6]:
def linear_interpolate_data(data):
    data = np.array(data)
    interpolate_values = np.arange(0,1,0.1)
    ret = []
    for i in range(0,len(data)-1):
        for j in interpolate_values:
            temp = data[i]*(1-j) + data[i+1] * j
            ret.append(temp.T)
    ret = np.array(ret)
    return ret

In [7]:
def add_padding(data, amt_to_pad, where_to_insert):
    data = np.array(data)
    padding = [data[where_to_insert]] * amt_to_pad
    data = np.insert(data, where_to_insert, padding, axis=0)
    return data

In [8]:
import itertools

In [9]:
def pad_rally_meta_data(data, amt_to_pad, num_strokes):
    data = np.array(data)
    temp = []
    for i in range(0, len(data)):
        temp.append([data[i]] * amt_to_pad)
    for i in range(0, len(temp)):
        data = np.insert(data, i*amt_to_pad, temp[i], axis=0)
    data = np.insert(data, len(data), data[-1], axis=0)
    data.resize((len(data) - num_strokes), 8)
    return np.array(data)

In [10]:
def extract_rally_data(data_extract, rally_id):
    player1 = []
    player2 = []
    ball = []
    rally_metadata = []
    data = data_extract.loc[events['rallyid'] == rally_id]
    data = data.reset_index()
    for i in range(0, len(data)):
        x1 = 0
        x2 = 0
        y1 = 0
        y2 = 0
        if i % 2 == 0:
            x1 = data.loc[i, 'hitter_x']
            y1 = data.loc[i, 'hitter_y']
            x2 = data.loc[i, 'receiver_x']
            y2 = data.loc[i, 'receiver_y']
            ball.append(np.asarray([x1,y1]))
        else:
            x2 = data.loc[i, 'hitter_x']
            y2 = data.loc[i, 'hitter_y']
            x1 = data.loc[i, 'receiver_x']
            y1 = data.loc[i, 'receiver_y']
            ball.append(np.asarray([x2, y2]))
        player1.append(np.asarray([x1, y1]))
        player2.append(np.asarray([x2, y2]))
        rally_metadata.append(["type "+str(data.loc[i, 'type']), "stroke "+str(data.loc[i, 'stroke']), 
                              "hitter "+str(data.loc[i, 'hitter']), "rallyid "+str(data.loc[i, 'rallyid']),
                              "strokeid "+str(data.loc[i, 'strokeid']), "server "+str(data.loc[i, 'server']),
                              "1st/2nd serve "+str(data.loc[i, 'serve_x']), "score "+str(data.loc[i, 'score'])])
    player1 = linear_interpolate_data(player1)
    player1 = add_padding(player1, 11, -1)
    player2 = linear_interpolate_data(player2)
    player2 = add_padding(player2, 11, -1)
    ball.append(np.asarray([data.loc[0, 'x'], data.loc[0, 'y']]))
    ball = linear_interpolate_data(ball)
    ball = np.insert(ball, len(ball), np.asarray([data.loc[0, 'x'], data.loc[0, 'y']]), axis=0)
    rally_metadata = pad_rally_meta_data(rally_metadata, 10, data.shape[0])
    return player1, player2, ball, rally_metadata

In [11]:
def create_figure(data, rally_id):
    player1, player2, ball, rally_metadata = extract_rally_data(data, rally_id)

    height_court = 10.97
    width_court = 11.89*2
    service_box = 6.4
    double_field = 1.37
    baseline_serviceline = 5.5
    breite_einzel = 8.23
    serviceline_net = 6.4
    colors=['red', 'cyan']

    # Create figure
    fig = go.Figure(
        data=[
            go.Scatter(x=[-10], y=[-10]),
            go.Scatter(x=[-10], y=[-10]),
            go.Scatter(x=[-10], y=[-10]),
            go.Scatter(x=[-10], y=[-10]),
            go.Scatter(x=[-10], y=[-10]),
            go.Scatter(x=[-10], y=[-10]),
            go.Scatter(x=[-10], y=[-10]),
            go.Scatter(x=[-10], y=[-10]),
            go.Scatter(x=[-10], y=[-10]),
            go.Scatter(x=[-10], y=[-10]),
            go.Scatter(x=[height_court, 0], y=[width_court/2, width_court/2],
                         mode="lines",
                         line=dict(width=2, color="white")),
              go.Scatter(x=[0,0, height_court, height_court, 0],
                         y=[0,width_court,width_court, 0, 0],
                         mode="lines",
                         line=dict(width=2, color="white")),
              go.Scatter(x=[height_court/2, width_court/2-service_box],
                         y=[height_court/2, width_court/2+service_box],
                        mode="lines",
                        line=dict(width=2, color="white")),
              go.Scatter(x=[height_court/2, height_court/2], y=[0, 0 + 0.45],
                        mode="lines",
                        line=dict(width=2, color="white")),
              go.Scatter(x=[height_court/2, height_court/2], y=[width_court, width_court - 0.45],
                mode="lines",
                line=dict(width=2, color="white")),
             go.Scatter(x=[1.37, 1.37], y=[0, width_court],
                mode="lines",
                line=dict(width=2, color="white")),
            go.Scatter(x=[height_court-1.37, height_court-1.37], y=[0, width_court],
                mode="lines",
                line=dict(width=2, color="white")),
            go.Scatter(x=[0+double_field,0+double_field, 
                          height_court-1.37, height_court-1.37, 0+double_field],
                       y=[baseline_serviceline, width_court/2+service_box, 
                          width_court/2+service_box, baseline_serviceline, baseline_serviceline],
                mode="lines",
                line=dict(width=2, color="white")),
            go.Scatter(
                 x=[-5],
                 y=[20],
                 mode='text',
                textfont=dict(color='white', size=12),
                 text=rally_metadata[0][7]),
            go.Scatter(
                 x=[-5],
                 y=[16],
                 mode='text',
                textfont=dict(color='white', size=12),
                 text=rally_metadata[0][6]),
                 go.Scatter(
                 x=[-5],
                 y=[18],
                 mode='text',
                textfont=dict(color='white', size=12),
                 text=rally_metadata[0][5]),
                 go.Scatter(
                 x=[-5],
                 y=[14],
                 mode='text',
                     textfont=dict(color='white', size=12),
                 text=rally_metadata[0][3])
              ],
        layout=go.Layout(
            xaxis=dict(range=[-9, height_court+2], autorange=False, zeroline=False, showgrid=False),
            yaxis=dict(range=[-6.5, width_court+6.5], 
                       autorange=False, zeroline=False, showgrid=False, ),
            height=800,
            width=550,
            title_text="Tennis Point",
            plot_bgcolor="#5080B0",
            updatemenus = [
        {
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 100, "redraw": True},
                                    "fromcurrent": True, "transition": {"duration": 300,
                                                                        "easing": "quadratic-in-out"}}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None],{"frame": {"duration": 0, "redraw": True},
                                      "mode": "immediate",
                                      "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": True,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }
    ]),

        frames=[go.Frame(
            data=[go.Scatter(
                x=[player1[k][0]],
                y=[player1[k][1]],
                mode="markers",
                marker=dict(color="red", size=10)),
                 go.Scatter(
                x=[player2[k][0]],
                y=[player2[k][1]],
                mode="markers",
                marker=dict(color="cyan", size=10)),
                  go.Scatter(
                x=[ball[k][0]],
                y=[ball[k][1]],
                mode="markers",
                marker=dict(color="yellow", size=10)),
                 go.Scatter(
                 x=[-5],
                 y=[6],
                 mode='text',
                     textfont=dict(color='white', size=12),
                 text=rally_metadata[k][0]),
                 go.Scatter(
                 x=[-5],
                 y=[8],
                 mode='text',
                     textfont=dict(color='white', size=12),
                 text=rally_metadata[k][1]),
                 go.Scatter(
                 x=[-5],
                 y=[10],
                 mode='text',
                     textfont=dict(color='white', size=12),
                 text=rally_metadata[k][2]),
                 go.Scatter(
                 x=[-5],
                 y=[12],
                 mode='text',
                     textfont=dict(color='white', size=12),
                 text=rally_metadata[k][4])])
            for k in range(len(player1))]
    )
    return fig

In [12]:
fig = create_figure(temp, 8)

In [13]:
fig.show()

In [14]:
player1, player2, ball, rally_metadata = extract_rally_data(temp, 8)

In [15]:
player1.shape

(71, 2)

In [16]:
rally_metadata.shape

(71, 8)

In [17]:
rally_metadata

array([['type serve', 'stroke forehand', 'hitter Nadal', 'rallyid 8',
        'strokeid 1', 'server Nadal', '1st/2nd serve second',
        'score 1:0, 15:0'],
       ['type serve', 'stroke forehand', 'hitter Nadal', 'rallyid 8',
        'strokeid 1', 'server Nadal', '1st/2nd serve second',
        'score 1:0, 15:0'],
       ['type serve', 'stroke forehand', 'hitter Nadal', 'rallyid 8',
        'strokeid 1', 'server Nadal', '1st/2nd serve second',
        'score 1:0, 15:0'],
       ['type serve', 'stroke forehand', 'hitter Nadal', 'rallyid 8',
        'strokeid 1', 'server Nadal', '1st/2nd serve second',
        'score 1:0, 15:0'],
       ['type serve', 'stroke forehand', 'hitter Nadal', 'rallyid 8',
        'strokeid 1', 'server Nadal', '1st/2nd serve second',
        'score 1:0, 15:0'],
       ['type serve', 'stroke forehand', 'hitter Nadal', 'rallyid 8',
        'strokeid 1', 'server Nadal', '1st/2nd serve second',
        'score 1:0, 15:0'],
       ['type serve', 'stroke forehand',

In [18]:
temp2 = temp.loc[temp['rallyid'] == 8]

In [19]:
temp2

,rallyid,frameid,strokeid,hitter,receiver,isserve,serve_x,type,stroke,hitter_x,...,server,returner,winner,reason,serve_y,strokes,totaltime,x,y,score
22,8,75885,1,Nadal,Djokovic,True,second,serve,forehand,5.48,...,Nadal,Djokovic,Djokovic,out,second,7.0,7.2,0.5,18.67,"1:0, 15:0"
23,8,75905,2,Djokovic,Nadal,False,first,return,backhand,3.82,...,Nadal,Djokovic,Djokovic,out,second,7.0,7.2,0.5,18.67,"1:0, 15:0"
24,8,75935,3,Nadal,Djokovic,False,first,topspin,forehand,5.49,...,Nadal,Djokovic,Djokovic,out,second,7.0,7.2,0.5,18.67,"1:0, 15:0"
25,8,75965,4,Djokovic,Nadal,False,first,topspin,backhand,7.78,...,Nadal,Djokovic,Djokovic,out,second,7.0,7.2,0.5,18.67,"1:0, 15:0"
26,8,76005,5,Nadal,Djokovic,False,first,topspin,backhand,7.80,...,Nadal,Djokovic,Djokovic,out,second,7.0,7.2,0.5,18.67,"1:0, 15:0"
27,8,76035,6,Djokovic,Nadal,False,first,topspin,forehand,1.54,...,Nadal,Djokovic,Djokovic,out,second,7.0,7.2,0.5,18.67,"1:0, 15:0"
28,8,76065,7,Nadal,Djokovic,False,first,topspin,forehand,2.72,...,Nadal,Djokovic,Djokovic,out,second,7.0,7.2,0.5,18.67,"1:0, 15:0"
